## Imports

In [ ]:
!pip install ../input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features

In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
import csv
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
import itertools
import cv2
import imageio
import numpy as np
import matplotlib.image as mpimg
import pandas as pd
import matplotlib.pylab as plt
from tqdm.auto import tqdm
# !pip install imageio-ffmpeg
import csv
import math
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import PIL
from PIL import Image, ImageDraw
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision.transforms as transforms
import torch.optim as optim
# !pip install utils
# import utils 
import os,sys
import IPython.display
import time
import glob
import matplotlib.pyplot as plt
import torchvision.transforms as T
from scipy.spatial import distance_matrix
import scipy.spatial.distance as scipy_distance
import scipy
import matplotlib

## Gathering Data

In [ ]:
n_test_videos = len(os.listdir('../input/nfl-health-and-safety-helmet-assignment/test/'))
# Run in debug mode unless during submission
if n_test_videos == 6:
    debug = True
else:
    debug = False

# debug = False

# Configurables
n_debug_samples = 1
random_state = 50
CONF_THRE = 0.3

# Read in the data.

BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')
    
tracking = add_track_features(tracking)

In [ ]:
#Add new columns using the video_frame column
def add_cols(df):
  if 'gameKey' not in df.columns:
    df['gameKey'] = df['video_frame'].str.split('_').str[0:1].str.join('_').astype(int)
  if 'playID' not in df.columns:
    df['playID'] = df['video_frame'].str.split('_').str[1:2].str.join('_').astype(int)
  if 'view' not in df.columns:
    df['view'] = df['video_frame'].str.split('_').str[2:3].str.join('_')
  if 'frame' not in df.columns:
    df['frame'] = df['video_frame'].str.split('_').str[3:4].str.join('_').astype(int)
  if 'video' not in df.columns:
      df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
  return df

helmets = add_cols(helmets)
#if debug:
if True:
  labels = add_cols(labels)
  # Select `n_debug_samples` worth of videos to debug with
  sample_videos = labels['video'].drop_duplicates()  #\
#      .sample(n_debug_samples, random_state=random_state).tolist() # Creates a list of n randomly selected videos
#   sample_videos = ['58106_002918_Endzone.mp4', '57786_003085_Sideline.mp4', '57992_000301_Sideline.mp4']

  sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos] # Creates a list of gameplay number in sample_videos

  #Removes all other video_frames that aren't in sample_videos or sample_gameplays
  tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
  helmets = helmets[helmets['video'].isin(sample_videos)]
  labels = labels[labels['video'].isin(sample_videos)]
labels.shape, tracking.shape, helmets.shape
display(labels)
display(tracking)
display(helmets)
print(sample_videos)

In [ ]:
# Remove helmets below a certain confidence threshold
helmets = helmets[helmets['conf']>CONF_THRE].copy()

# If there are still more than 22 helmets, keep only the best 22 helmets with highest confidence
low_conf_helms_removed = []
for vf in helmets['video_frame'].unique():
  helm = helmets[helmets['video_frame'] == vf]
  if len(helm) > 22:
    helm = helm.sort_values('conf', ascending = False)
    low_conf_helms_removed.append(helm.head(22))
    continue
  low_conf_helms_removed.append(helm)

helmets = pd.concat(low_conf_helms_removed)
helmets['x']=helmets['left']+helmets['width']/2
helmets['y']=helmets['top']+helmets['height']/2

In [ ]:
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
def dist(v1, v2):
  # print(v1)
  return np.sqrt(np.square(v1[0] - v2[0]) + np.square(v1[1] - v2[1]))

def get_NN_borders(list_points, amount=5):
  results = np.zeros((4, amount)) #0xmin, 1ymin, 2xmax, 3ymax
  lpmax = np.copy(list_points)
  lpmin = np.copy(list_points)
  for i in range(amount):
    results[:2,i]=np.argmin(lpmin, 0)
    #print(results[0,i])
    lpmin[int(results[0,i])][0] = 1000000
    lpmin[int(results[1,i]), 1] = 1000000
    results[2:,i]=np.argmax(lpmax, 0)
    lpmax[int(results[2,i]), 0],lpmax[int(results[3,i]), 1] = -np.inf, -np.inf 
  return results

def create_combinations(track):
  mesh = np.array(np.meshgrid(track[0], track[1], track[2], track[3]))
  combinations = mesh.T.reshape(-1, 4)
  c = 0
  for index, i in enumerate(combinations):
    if i[0]==i[1] or i[0]==i[2] or i[0]==i[3] or i[1]==i[2] or i[1]==i[3] or i[2]==i[3]:
      combinations = np.delete(combinations, int(index)-c, 0)
      c+=1
  return combinations
def convert_data_tonumpy(helm, track, view):
  if 'Endzone'==view:
    helm['x']=1280-helm['x']
  helm['y']=720-helm['y']
  tracking_points = track[['x','y']].to_numpy()
  if 'Sideline'==view:
    helmet_points = helm[['x','y']].to_numpy()
  else:
    helmet_points = np.c_[helm['y'].to_numpy(), helm['x'].to_numpy()]
  return helmet_points, tracking_points

## Get deepsort clusters

In [ ]:
import sys
sys.path.append('../input/easydict-master/easydict-master/')
# https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch
sys.path.append('../input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch/')
from deep_sort.deep_sort import DeepSort
from utils.parser import get_config

In [ ]:
%%writefile deepsort.yaml

DEEPSORT:
  REID_CKPT: "../input/yolov5-deepsort-pytorch/ckpt.t7"
  MAX_DIST: 0.2
  MIN_CONFIDENCE: 0.3
  NMS_MAX_OVERLAP: 0.5
  MAX_IOU_DISTANCE: 0.9
  MAX_AGE: 15
  N_INIT: 1
  NN_BUDGET: 30

In [ ]:
"""
Helper functions from yolov5 to plot deepsort labels.
"""

def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    #A contiguous array is just an array stored in an unbroken block of memory
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    #Generate corner points for making rectangle
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #Put label on the bounding box if available
    if label: 
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im

In [ ]:
def deepsort_helmets(video_data,
                     video_dir,
                     deepsort_config='deepsort.yaml',
                     plot=False,
                     plot_frames=[]):
    
    # Setup Deepsort
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,
                        max_dist=cfg.DEEPSORT.MAX_DIST,
                        min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                        max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,
                        nn_budget=cfg.DEEPSORT.NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True)
    ds = []
    output_list = []
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):
        #Calculate center points
        d['x'] = (d['left'] + round(d['width'] / 2))
        d['y'] = (d['top'] + round(d['height'] / 2))
        

        xywhs = d[['x','y','width','height']].values
        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}')

        video_frame = myvideo.replace(".mp4", "") + '_' + str(frame)
#         print(video_frame)
        #Retrieve correct frame from the video
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        success, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #Get the confidence and class for each player in the frame
        confs = np.ones([len(d),])
        clss =  np.zeros([len(d),])
        outputs = deepsort.update(xywhs, confs, clss, image)
        outputs_with_videoframe = []
        for idx, op in enumerate(outputs.copy()):
          op[2], op[1] = op[1], op[2]  # Switch top and right
          op[1] = op[1] - op[0] # Width = right - left
          op[3] = op[3] - op[2] #Height = bottom - top
          row = [op[0], op[1], op[2], op[3], op[4], video_frame]
          outputs_with_videoframe.append(row)
        
        header = ['left','width','top','height','deepsort_cluster', 'video_frame']
#         print(outputs_with_videoframe)
        if frame > 1:
            preds_df_with_videoframe = pd.DataFrame(outputs_with_videoframe, columns = header)
            output_list.append(preds_df_with_videoframe)

        #Plot specified frames
        if (plot and frame > cfg.DEEPSORT.N_INIT) or (frame in plot_frames):
            for j, (output, conf) in enumerate(zip(outputs, confs)): 

                bboxes = output[0:4]
                id = output[4]
                cls = output[5]

                c = int(cls)  # integer class
                label = f'{id}'
                color = compute_color_for_id(id)
                im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
            fig, ax = plt.subplots(figsize=(15, 10))
            video_frame = d['video_frame'].values[0]
            ax.set_title(f'Deepsort labels: {video_frame}')
            plt.imshow(im)
            plt.show()

        #Create a dataframe for predictions
        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom', 'class','deepsort_cluster'])
        if len(preds_df) > 0:
            # TODO Fix this messy merge
            d = pd.merge_asof(d.sort_values(['left','top']),
                              preds_df[['left','top','deepsort_cluster']] \
                              .sort_values(['left','top']), on='left', suffixes=('','_deepsort'),
                              direction='nearest')
        ds.append(d)
        # display(d)
    #Generate a final dataframe for class predictions over all frames    
    dout = pd.concat(ds)
    output_df = pd.concat(output_list)
    return output_df


In [ ]:
if debug:
    video_dir = '../input/nfl-health-and-safety-helmet-assignment/train/'
else:
    video_dir = '../input/nfl-health-and-safety-helmet-assignment/test/'
    
# print(glob.glob(video_dir))

# Uncomment this when trying a new video
# Deletes the existing deepsort clusters file to create a new one.
# if os.path.isfile('/content/drive/MyDrive/Team 3/NFL-Health-Safety/helmet-assignment/deepsort_clusters_submission.csv'):
#   os.remove('/content/drive/MyDrive/Team 3/NFL-Health-Safety/helmet-assignment/deepsort_clusters_submission.csv')
output = []
for myvideo, video_data in tqdm(helmets.groupby('video'), total= helmets['video'].nunique()):
    print(f'==== {myvideo} ====')
    if debug:
        # Plot deepsort labels when in debug mode.
        out = deepsort_helmets(video_data, video_dir, plot_frames=[10, 50, 100, 150,200,250,300,350])
    else:
        out = deepsort_helmets(video_data, video_dir)
    output.append(out)

output = pd.concat(output)
output.to_csv('deepsort_clusters.csv', index = False, mode='w')

In [ ]:
# output

## Getting Homography

In [ ]:
def flip_view(helmets, args = 2):
  
  if args == 2:
    for ii, (x,y) in enumerate(helmets):
      helmets[ii][0] = 1280 - x
      helmets[ii][1] = 720  - y
  return helmets

In [ ]:
def flip_view_dataframe(deepsort_helms):
  
  deepsort_helms['x'] = 1280 - deepsort_helms['x']
  deepsort_helms['y'] = 720 - deepsort_helms['y']

  return deepsort_helms

In [ ]:
'''
@input : train_gt: detected helmet locations that you want to project
         track_gt: tracking data of the helmets in the same frame
         view    : Sideline or Endzone
@return: h      : best estimated homography matrix
'''
def getHomographyTryExact(train_gt, track_gt, view): #gameK, playid, videoname, train_gt, track_gt, frame_number, view):
  time1 = time.time()
  
  #Check if the view is from one side of the field or another.
  view_flip = False
  #adapt the coordinates depending on the view  
  if 'Endzone'==view:
    train_gt['x']=1280-train_gt['x']
  train_gt['y']=720-train_gt['y']
  tracking_points = track_gt[['x','y']].to_numpy()
  if 'Sideline'==view:
    helmet_points = train_gt[['x','y']].to_numpy()
  else:
    helmet_points = np.c_[train_gt['y'].to_numpy(), train_gt['x'].to_numpy()]
  amount_players_tracking, amount_players_helmets = len(tracking_points), len(helmet_points)
  tracking_borders_index= get_NN_borders(tracking_points, max(amount_players_tracking-amount_players_helmets,5)) #0xmin, 1ymin, 2xmax, 3ymax
  helmet_borders_index = get_NN_borders(helmet_points, 2)
  #create combinations of the border points
  list_combinations_tracking = create_combinations(tracking_borders_index)
  list_combinations_helmet = create_combinations(helmet_borders_index)

  print('Length combinations tracking', len(list_combinations_tracking))
  print('Length combinations helmet', len(list_combinations_helmet))

  current_combination = np.zeros((4,2))
  helmet_borders = np.zeros((4,2))
  bestH = None
  bestPoints = None
  bestHelm = None
  min=np.inf
  for ii in range(2):
    if ii == 1:
      helmet_points = flip_view(helmet_points.copy())
      helmet_borders_index = get_NN_borders(helmet_points, 2)
      list_combinations_helmet = create_combinations(helmet_borders_index)

    for j, comb_helmets in enumerate(list_combinations_helmet):

        for q in range(4):
          helmet_borders[q]=helmet_points[int(comb_helmets[q])]

        for i, comb in enumerate(list_combinations_tracking):
          current_combination[0],current_combination[1],current_combination[2],current_combination[3] = tracking_points[int(comb[0])],tracking_points[int(comb[1])],tracking_points[int(comb[2])],tracking_points[int(comb[3])]
          h, status = cv2.findHomography(helmet_borders,current_combination)
          if h is not None:
            helmet_points_transformed = cv2.perspectiveTransform(np.array([helmet_points]), h)
            dist_matrix =scipy.spatial.distance.cdist(helmet_points_transformed[0], tracking_points)
            row_ind, col_ind = scipy.optimize.linear_sum_assignment(dist_matrix)
            count_min = dist_matrix[row_ind, col_ind].sum()
            if count_min<min :
              bestPoints = np.copy(helmet_points_transformed)
              bestHelm = np.copy(helmet_points)
              best_combination = np.copy(current_combination)
              print('New min:', count_min, 'view flip:', view_flip)
              min = count_min
              bestH = np.copy(h)
              if ii == 1:
                view_flip = True
          else:
            print('h is none')
    print('BEST MIN',min)

  
  ###########################
  dist_matrix =scipy.spatial.distance.cdist(bestPoints[0], tracking_points)
  row_ind, col_ind = scipy.optimize.linear_sum_assignment(dist_matrix)
  count_min = dist_matrix[row_ind, col_ind].sum()
  c = 0  
  print("View Flip:", view_flip)
  print('Accuracy:',c,'/',len(row_ind))
  print('time for one frame:', time.time()-time1)
  print('rowind', row_ind, '//', len(row_ind))
  print('col_ind', col_ind, '//',len(col_ind))
  plt.figure(figsize=(10,10))
  a = plt.plot(bestPoints[0,:,0] , bestPoints[0,:,1],'bo', markersize = 10) #.set_label("Estimation of helmets on map") #bleu
  b = plt.plot(tracking_points[:,0], tracking_points[:,1],'rs',  markersize = 7, label= "Tracking GPS") #red
  plt.show()
  return bestH, view_flip

'''
@input : helmets_data: detected helmet locations of the entire video
         tracking_data: tracking data of the entire video
         view    : Sideline or Endzone
@return: list_homo: a list of all the estimated homography matrices
'''

def findHomography_with_one_iteration(arranged_helm_points,arranged_track_points, Hprev):
    #NewH1, status = cv2.findHomography(arranged_helm_points[:,:2],arranged_track_points[:,:2], method =0) # cv2.LMEDS)
    # if view_flip:
    #   arranged_helm_points = flip_view(arranged_helm_points)
    helm_trans0 = cv2.perspectiveTransform(np.array([arranged_helm_points[:,:2]],dtype=np.float32), Hprev)
    distances = np.zeros(len(arranged_helm_points))
    thresh = int(len(arranged_helm_points)*0.5)
    for r in range(len(helm_trans0[0])):
      #print('lkjl', scipy.spatial.distance.euclidean(helm_trans0[0,r,:2], arranged_track_points[r,:2]))
      distances[r]=scipy.spatial.distance.euclidean(helm_trans0[0,r,:2], arranged_track_points[r,:2])
    dist_ind_ok = np.argwhere(distances<1)
    if len(dist_ind_ok)<thresh or len(dist_ind_ok)<7:
      # print('NEED MORE PAIRS ONE ITERATION')
      c=1
      while (len(dist_ind_ok)<thresh or len(dist_ind_ok)<7) and c<100:
        c+=0.1
        dist_ind_ok = np.argwhere(distances<c)
      # print('C', c)
    NewH2, status = cv2.findHomography(arranged_helm_points[dist_ind_ok,:2],arranged_track_points[dist_ind_ok,:2], method =0) 
    return NewH2

def find_index_array(arranged_track_points, dup1):
  indexes= np.empty(0, np.int)
  for i in range(len(dup1)):
    indexes= np.append(indexes,np.where((arranged_track_points==dup1[i]).all(axis=1))[0])
  return indexes

In [ ]:
def homography_for_all_frames(videoname, video_path, helmets_deepsort, tracking_data, show_plots = False):
  #helmets_deepsort = preprocessing_with_occurences_deepsort_output(helmets_deepsort, 29)
  vn = videoname.split('_')
  gameK=int(vn[0])
  playid = int(vn[1])
  view = str(vn[2]).replace('.mp4', '')
  video_filename = video_path+videoname
  num_frames = helmets_deepsort['frame'].nunique() + 1
  print(num_frames)
  videoname = videoname.replace('.mp4','')
  list_of_homography = np.zeros((num_frames, 3,3))
  labels_clusters = np.zeros(max(helmets_deepsort['deepsort_cluster'])).astype('U256')
  #HOMOGRAPHY FIRST IMAGE
  helm_points = helmets_deepsort[(helmets_deepsort['video_frame']==videoname+'_'+str(2))]
  track_points = tracking_data[(tracking_data['gameKey']==gameK) & (tracking_data['playID']==playid)]
  est_frame = find_nearest(track_points.est_frame.values, 2)
  track_points = track_points[track_points['est_frame']==est_frame].reset_index()
  homography, view_flip = getHomographyTryExact(helm_points.copy(), track_points.copy(), view)
  list_of_homography[1]=homography
  list_of_homography[0]=homography

  #CONVERT X Y OF IMAGE TO SATISFY VIEW
  if 'Endzone'==view:
    helmets_deepsort['x']=1280-helmets_deepsort['x']
  helmets_deepsort['y']=720-helmets_deepsort['y']
  if 'Endzone'==view:
    X = helmets_deepsort['y'].copy()
    Y = helmets_deepsort['x'].copy()
    helmets_deepsort['x']=X
    helmets_deepsort['y']=Y
    
  if view_flip:
    helmets_deepsort = flip_view_dataframe(helmets_deepsort.copy())
    
  helm_points = helmets_deepsort[(helmets_deepsort['video_frame']==videoname+'_'+str(2))]
#   track_points = track_points[(tracking_data['gameKey']==gameK) & (tracking_data['playID']==playid)]

#   helm, track = convert_data_tonumpy(helm_points, track_points, view)
  track = track_points[['x','y']].to_numpy()
  helm = helm_points[['x','y']].to_numpy()
  print("Helm and Track Lengths:",len(helm), len(track))
  
  helmet_points_transformed = cv2.perspectiveTransform(np.array([helm]), homography)
  dist_matrix =scipy.spatial.distance.cdist(helmet_points_transformed[0], track)
#   print(dist_matrix)
  row_ind, col_ind = scipy.optimize.linear_sum_assignment(dist_matrix)
  count_min = dist_matrix[row_ind, col_ind].sum()
  # print('row_ind', row_ind, 'col_ind', col_ind)
  for i in range(len(row_ind)):
    labels_clusters[i]=track_points['player'][col_ind[i]]
  #print(labels_clusters)
    
    
  #display(helmets_deepsort)
  #print('numebrframes', num_frames)
  for frame_number in range(1,num_frames-1):
    helm_points = helmets_deepsort[(helmets_deepsort['video_frame']==videoname+'_'+str(frame_number + 2))].reset_index()
    track_points = tracking_data[(tracking_data['gameKey']==gameK) & (tracking_data['playID']==playid)]
    est_frame = find_nearest(track_points.est_frame.values, frame_number+2)
    track_points = track_points[track_points['est_frame']==est_frame].reset_index()
    t = track_points[['x','y']].to_numpy()
    h = helm_points[['x','y']].to_numpy()
    arranged_track_points = np.empty((0,2), np.float32)
    arranged_helm_points = np.empty((0,3), np.float32)
    assign_new_label = np.empty((0,3), np.float32) #x, y, cluster

    for i in range(len(helm_points)):
      if track_points[track_points['player']==labels_clusters[helm_points['deepsort_cluster'][i]-1]].empty:
        assign_new_label = np.append(assign_new_label, np.array([[helm_points['x'][i],helm_points['y'][i],int(helm_points['deepsort_cluster'][i])]]), axis=0)
      else:
        arranged_helm_points = np.append(arranged_helm_points, np.array([[helm_points['x'][i],helm_points['y'][i],int(helm_points['deepsort_cluster'][i])]]), axis=0)
        arranged_track_points = np.append(arranged_track_points, track_points[track_points['player']==labels_clusters[helm_points['deepsort_cluster'][i]-1]][['x','y']].to_numpy(), axis=0)

    u1, c1 = np.unique(arranged_track_points, return_counts=True,axis=0)
    dup1 = u1[c1 > 1]
    # print('-----------------------------------------------------------------------------------------')
    helmets_with_no_label = np.empty((0,3), np.float32)

    #Look if there are duplicates in the tracking points
    location = find_index_array(arranged_track_points, dup1)
    if len(location)>0:
      #print(arranged_track_points)
      #print(location)
      duplicate_helmets = arranged_helm_points[location]
      duplicate_tracking = arranged_track_points[location]
      helmets_with_no_label = np.append(helmets_with_no_label, duplicate_helmets,axis=0)
      arranged_track_points = np.delete(arranged_track_points, location, 0)
      arranged_helm_points = np.delete(arranged_helm_points, location, 0)
      #print(arranged_track_points)
      #print('aa',helmets_with_no_label)
    
    #Add assign new label to the list that need to be reconverted
    helmets_with_no_label = np.append(helmets_with_no_label, assign_new_label[:,:],axis=0)
    #NewH, status = cv2.findHomography(arranged_helm_points[:,:2],arranged_track_points[:,:2], method =0) # cv2.LMEDS)
    # print(arranged_helm_points)
#     if view_flip:
#       arranged_helm_points = flip_view(arranged_helm_points, args = 3)
    # print(arranged_helm_points)
    NewH = findHomography_with_one_iteration(arranged_helm_points,arranged_track_points,list_of_homography[frame_number-1])
    # print('bb',helmets_with_no_label)
    
    #Look if there are helmet-tracking pairs that have a big distance
    locations_to_del=np.empty(0, np.int)
#     print(arranged_helm_points)
    helm_trans1 = cv2.perspectiveTransform(np.array([arranged_helm_points[:,:2]],dtype=np.float32), NewH)
    for r in range(len(helm_trans1[0])):
      if scipy.spatial.distance.euclidean(helm_trans1[0,r,:2], arranged_track_points[r,:2])>5:
        locations_to_del=np.append(locations_to_del, r)
        # print('arranged_helm_points[r,:2]',arranged_helm_points[r],arranged_helm_points[r].shape)
        helmets_with_no_label = np.append(helmets_with_no_label,np.array([arranged_helm_points[r]]), axis=0)
    if len(locations_to_del)>0:
      # print('lllll', locations_to_del)
      arranged_track_points = np.delete(arranged_track_points, locations_to_del, 0)
      arranged_helm_points = np.delete(arranged_helm_points, locations_to_del, 0)
    # print('cc',helmets_with_no_label)
    
    #NewH, status = cv2.findHomography(arranged_helm_points[:,:2],arranged_track_points[:,:2], method =0)
    NewH = findHomography_with_one_iteration(arranged_helm_points,arranged_track_points,list_of_homography[frame_number-1])
    if NewH is not None:
      list_of_homography[frame_number+1] = NewH
    if len(helmets_with_no_label)>0 and NewH is not None:
        # print('helmets_with_no_label', np.array([helmets_with_no_label]).shape, np.array([helmets_with_no_label]))
        h_no_label = cv2.perspectiveTransform(np.array([helmets_with_no_label[:,:2]],dtype=np.float32), NewH)
        t_without_alreadyassigned = np.copy(t)
        for q in range(len(t_without_alreadyassigned)):
          for r in range(len(arranged_track_points)):
            if t_without_alreadyassigned[q][0]== arranged_track_points[r,0] and t_without_alreadyassigned[q][1]== arranged_track_points[r,1]:
              t_without_alreadyassigned[q]=np.array([100000,100000])
        dist_matrix =scipy.spatial.distance.cdist(h_no_label[0], t_without_alreadyassigned)
        row_ind, col_ind = scipy.optimize.linear_sum_assignment(dist_matrix)
        #print('row_ind', row_ind)
        #print('col_ind', col_ind)
        for r in range(len(col_ind)):
          labels_clusters[int(helmets_with_no_label[r,2]-1)]=track_points['player'][col_ind[r]]
    
    if frame_number%50==0 and show_plots:
        #print(labels_clusters)
        arr_helm_points_trans = cv2.perspectiveTransform(np.array([arranged_helm_points[:,:2]],dtype=np.float32), NewH)
        h_trans = cv2.perspectiveTransform(np.array([arranged_helm_points[:,:2]],dtype=np.float32), NewH)
#         img = vid.get_data(frame_number +1)
#         plt.imshow(img)
#         plt.show()

        #print('arranged_track_points',arranged_track_points)
        #print('arr_helm_points_trans',arr_helm_points_trans)
        """
        plt.figure(figsize=(10,10))
        a = plt.plot(arranged_helm_points[:,0], arranged_helm_points[:,1],'bo', markersize = 10) #.set_label("Estimation of helmets on map") #bleu
        b = plt.plot(h[:,0], h[:,1],'rs',  markersize = 7, label= "Tracking GPS") #red
        plt.show()
        """
        
        plt.figure(figsize=(10,10))
        if len(helmets_with_no_label)>0:
          new_trans = cv2.perspectiveTransform(np.array([helmets_with_no_label[:,:2]],dtype=np.float32), NewH)
          c = plt.plot(new_trans[0,:,0], new_trans[0,:,1],'gs',  markersize = 12, label= "Tracking GPS") #red

        a = plt.plot(h_trans[0,:,0], h_trans[0,:,1],'bo', markersize = 10) #.set_label("Estimation of helmets on map") #bleu
        b = plt.plot(t[:,0], t[:,1],'rs',  markersize = 7, label= "Tracking GPS") #red
#         for p in range(len(arranged_track_points)):
#           plt.plot([arr_helm_points_trans[0,p,0], arranged_track_points[p,0]], [arr_helm_points_trans[0,p,1], arranged_track_points[p,1]], 'k')

#         plt.axes().set_aspect('equal')
        #plt.legend([a,b], ["Estimation of helmets on map","Tracking GPS"])
#         red_patch = matplotlib.patches.Patch(color='red', label='Tracking GPS')
#         blue_patch = matplotlib.patches.Patch(color='blue', label='Estimation of helmets on map')
        #green_patch = matplotlib.patches.Patch(color='green', label='Corner helmets used')
#         plt.legend(handles=[red_patch,blue_patch])
        plt.show()
        
        print(frame_number, '-------------------')
  return labels_clusters, list_of_homography, view_flip

## Assigning labels to clusters

In [ ]:
def update_distmatrix(dist_matrix, comb_matrix_clustrack, track_ind, helm_clusters, type_clusters, type_players, final_results):
  ii = np.empty((1,0), np.int)
  jj = np.empty((1,0), np.int)
  for i in range(len(dist_matrix)):
    for j in range(len(dist_matrix[0])):
      a = comb_matrix_clustrack[helm_clusters[i]==type_clusters, track_ind[j]==type_players]
      b=dist_matrix[i][j]
      dist_matrix[i][j]=dist_matrix[i][j]+comb_matrix_clustrack[helm_clusters[i]==type_clusters, track_ind[j]==type_players]*100000000
      if final_results.at[helm_clusters[i], 'PlayerID' ] == track_ind[j]:
        ii = np.append(ii, i)
        jj = np.append(jj, j)
  if len(ii)>0:
    dist_matrix = np.delete(dist_matrix, jj, 1)
    dist_matrix = np.delete(dist_matrix, ii, 0)
  return dist_matrix, ii, jj

In [ ]:
def create_comb_matrix_clusclus(output_deepsort):
  #display(output_deepsort)
  type_clusters= output_deepsort['deepsort_cluster'].value_counts().index.to_numpy()
  combMatrix = np.zeros((len(type_clusters), len(type_clusters)))
  for i in range(len(type_clusters)):
    for j in range(len(type_clusters)):
      od1 = output_deepsort[(output_deepsort['deepsort_cluster']==type_clusters[i]) | (output_deepsort['deepsort_cluster']==type_clusters[j])]
      has_duplicate_labels = od1[["frame"]].duplicated().any()
      if has_duplicate_labels:
        combMatrix[i, j]=1 
      if i==j:
        combMatrix[i, j]=1 
  #display(combMatrix)
  return combMatrix


In [ ]:
#inp = final res: updated final results
#       comb_matrix_clusclus,
#       comb_matrix_clustrack to update
#       index_type_player
#       index_cluster
def update_comb_matrix_clustrack(final_results, comb_matrix_clusclus, comb_matrix_clustrack,index_type_player, index_cluster):
  for i in range(len(comb_matrix_clustrack)):
    if comb_matrix_clusclus[index_cluster][i]==1:
      comb_matrix_clustrack[i][index_type_player]= 1
  return comb_matrix_clustrack
def flip_view_dataframe(deepsort_helms):
  deepsort_helms['x'] = 1280 - deepsort_helms['x']
  deepsort_helms['y'] = 720 - deepsort_helms['y']
  return deepsort_helms

In [ ]:
########################################################

def Majority_votingV7(Homo_list, output_deepsort, tracking_data, videoname, view_flip,comb_matrix_clusclus, current_results):

  out_deep = output_deepsort.copy()
  vn = videoname.split('_')
  gameK=int(vn[0])
  playid = int(vn[1])
  view = str(vn[2]).replace('.mp4', '')
  video_filename = video_path+videoname
  #vid = imageio.get_reader(video_filename, 'ffmpeg')
  #num_frames=vid._meta['nframes']
  videoname = videoname.replace('.mp4','')
  if 'Endzone'==view:
    output_deepsort['x']=1280-output_deepsort['x']
  output_deepsort['y']=720-output_deepsort['y']
  if 'Endzone'==view:
    X = output_deepsort['y'].copy()
    Y = output_deepsort['x'].copy()
    output_deepsort['x']=X
    output_deepsort['y']=Y
  if view_flip:
    output_deepsort = flip_view_dataframe(output_deepsort.copy())

  track_points = tracking_data[(tracking_data['playID']==playid)]
  count_clusters = output_deepsort['deepsort_cluster'].value_counts().to_numpy()
  type_clusters= output_deepsort['deepsort_cluster'].value_counts().index.to_numpy()
  type_players = track_points['player'].unique()
  print('count_clusters',count_clusters)
  print('type of clusters',type_clusters)
  print(track_points['player'].unique())

  final_results= current_results.copy()
  colname=['gameKey', 'playID', 'player', 'time', 'x', 'y', 's', 'a', 'dis', 'o','dir', 'event', 'game_play', 'snap', 'isSnap', 'team', 'snap_offset','est_frame', 'frame_num']
  all_track_points = pd.DataFrame(None, columns = colname) #.insert(len(track_points.columns),'Frame_num',
  for q in range(2,len(Homo_list)+1):
    est_frame = find_nearest(track_points.est_frame.values, q)
    track = track_points[track_points['est_frame']==est_frame].reset_index()
    track['frame_num']=q
    all_track_points = all_track_points.append(track)
  #output_deepsort['frame']=output_deepsort['video_frame'].map(lambda element: int(element.split('_')[-1]))
  display(all_track_points)
  all_track_points.drop('index', 1, inplace=True)
  all_track_points.reset_index(inplace=True)
  comb_matrix_clustrack = np.zeros((len(type_clusters), len(type_players)))
  for i in range(len(final_results)):
    if final_results.at[type_clusters[i],'PlayerID' ] != '0.0':
      index_type_player = np.where(type_players==final_results.at[type_clusters[i],'PlayerID' ])
      comb_matrix_clustrack = update_comb_matrix_clustrack(final_results, comb_matrix_clusclus, comb_matrix_clustrack,index_type_player, i)

  for cluster in range(len(type_clusters)):
    #for cluster in range(2):

    if final_results.at[type_clusters[cluster],'PlayerID' ] =='0.0':
      #print('in1')
      results = pd.DataFrame( np.zeros((len(type_clusters),len(type_players))))
      results.index=type_clusters
      results.columns = type_players
      cnone = 0
      cnotnone=0
      for frame_num in range(2,len(Homo_list)):
        track = all_track_points[all_track_points['frame_num']==frame_num].copy()
        helm = output_deepsort[(output_deepsort['frame']==frame_num)].copy() #need to finish the location of the first players on first frame
        track_ind = track['player'].to_numpy()
        helm_clusters = helm['deepsort_cluster'].to_numpy()
        t = track[['x','y']].to_numpy()
        h = helm[['x','y']].to_numpy()
        if h is not None or t is not None:
          h_trans = cv2.perspectiveTransform(np.array([h[:,:2]],dtype=np.float32), Homo_list[frame_num-1])
          if (h_trans is None) or (t is None) or (h is None):
            cnone+=1
          else:
            cnotnone+=1
            dist_matrix =scipy.spatial.distance.cdist(h_trans[0], t)
            dist_matrix_update, deleted_clus , deleted_track = update_distmatrix(dist_matrix, comb_matrix_clustrack, track_ind, helm_clusters, type_clusters, type_players, final_results)
            if len(dist_matrix)>0:
              row_ind, col_ind = scipy.optimize.linear_sum_assignment(dist_matrix_update) #row: indexes of helmets, col: indexes of tracking
            #if cluster>70:
            #  print(dist_matrix_update)
              
              helm_clusters = np.delete(helm_clusters, deleted_clus, 0)
              track_ind = np.delete(track_ind, deleted_track,0)
              for r in range(len(row_ind)):             
                results.at[int(helm_clusters[row_ind[r]]),track_ind[col_ind[r]]]=results.at[int(helm_clusters[row_ind[r]]),track_ind[col_ind[r]]]+1
      for i in range(len(final_results)):
        if final_results.at[type_clusters[i], 'PlayerID' ] != '0.0':
          a= len(results)
          results = results.drop(type_clusters[i])
          #print('remooooooooove',final_results.at[type_clusters[i], 'PlayerID' ] , 'before:', a, 'after', len(results))
      maxxx = results.max()
      col_max = maxxx.idxmax(axis=0, skipna=True) #player id
      row_max = results[col_max].idxmax(skipna=True) #cluster 
      final_results.at[int(row_max),'PlayerID' ] = col_max
      print(cluster,'New combination:',col_max, 'with', row_max)
      ind_player = np.where(type_players==col_max)
      ind_clus = np.where(type_clusters==row_max)
      print('ind_palyer', int(ind_player[0]), 'ind_clus', int(ind_clus[0]))
      comb_matrix_clustrack = update_comb_matrix_clustrack(final_results, comb_matrix_clusclus, comb_matrix_clustrack,int(np.where(type_players==col_max)[0]), int(np.where(type_clusters==row_max)[0]))
      
        
        
      #with np.printoptions(threshold=np.inf):
      #  print('comb_matrix_clustrack')
      #  print(comb_matrix_clustrack)
      
  display(final_results)
  return final_results

## Making a submission

In [ ]:
# sample_videos

In [ ]:
#if debug:
#    video_path = '.../input/NFL-Health-Safety/nfl-health-and-safety-helmet-assignment/train/'
#else:
#    video_path = '.../input/NFL-Health-Safety/nfl-health-and-safety-helmet-assignment/test/'
video_path = '.../input/NFL-Health-Safety/nfl-health-and-safety-helmet-assignment/train/'
#helmets_deepsort = pd.read_csv('deepsort_clusters.csv')
helmets_deepsort = pd.read_csv("../input/deepsort-clusters5/deepsort_clusters.csv")


# helmets_deepsort = pd.read_csv('../input/ds-clusters/deepsort_clusters.csv')
# print(helmets_deepsort['deepsort_cluster'].unique())
helmets_deepsort = add_cols(helmets_deepsort)
helmets_deepsort['deepsort_cluster'] = helmets_deepsort['deepsort_cluster'].astype('int')
helmets_deepsort['x']=helmets_deepsort['left']+helmets_deepsort['width']/2
helmets_deepsort['y']=helmets_deepsort['top']+helmets_deepsort['height']/2

submission_df = []
header = ['video_frame', 'label', 'left', 'width', 'top', 'height']
print(helmets_deepsort['video'].unique())
for vvv in range(len(helmets_deepsort['video'].unique())):
  if vvv>0:
    continue
  vid=helmets_deepsort['video'].unique()[vvv]
  print("Video:", vid)
  
#   if '57995_000109_Endzone.mp4' not in vid:
#     continue

  gameK  = int(vid.split('_')[0])
  playID = int(vid.split('_')[1])
  view   = (vid.split('_')[2]).split('.')[0]
  #cluster_info = sort_by_span(find_cluster_span(vid,helmets_deepsort))
  #if debug:
  #    display(cluster_info)

  tracking_one_vid = tracking[(tracking['gameKey'] == gameK) & (tracking['playID'] == playID)].reset_index()
  helmets_deepsort_one_vid = helmets_deepsort[helmets_deepsort['video'] == vid].reset_index()
    
  labels_clusters, list_of_homography, view_flip = homography_for_all_frames(vid, video_path, helmets_deepsort_one_vid.copy(), tracking.copy())
  display(helmets_deepsort_one_vid)
  print('---------------------assigning cluster to player----------------------------')
  comb_matrix_clusclus = create_comb_matrix_clusclus(helmets_deepsort_one_vid.copy())
  print(comb_matrix_clusclus)
  type_clusters= helmets_deepsort_one_vid['deepsort_cluster'].value_counts().index.to_numpy()
  display(type_clusters)
  display(helmets_deepsort_one_vid['deepsort_cluster'].value_counts().to_numpy())
  final_results = pd.DataFrame( np.zeros((len(type_clusters),1)).astype(np.unicode_), index=type_clusters, columns=['PlayerID'])
  
  Majority_voting_result = Majority_votingV7(list_of_homography, helmets_deepsort_one_vid.copy(), tracking_one_vid.copy(), vid, view_flip,comb_matrix_clusclus.copy(), final_results)
  print('-----------------adding cluster columns--------------------------')
  Majority_voting_result['cluster']=Majority_voting_result.index
  with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(Majority_voting_result)
  print('-------------------associating helmets with players id--------------------------')
  columns = ['video_frame', 'left', 'width', 'top', 'height', 'label']
  sub_video = []
  for cluster in helmets_deepsort_one_vid['deepsort_cluster'].unique():
    deep_sort_cluster_data = helmets_deepsort_one_vid[helmets_deepsort_one_vid['deepsort_cluster'] == cluster].reset_index()
    player_label = Majority_voting_result[Majority_voting_result['cluster'] == cluster].reset_index()['PlayerID'][0]
    for idx, vf in enumerate(deep_sort_cluster_data['video_frame']):
        left = deep_sort_cluster_data['left'][idx]
        width= deep_sort_cluster_data['width'][idx]
        top = deep_sort_cluster_data['top'][idx]
        height = deep_sort_cluster_data['height'][idx]
        row = [vf, left, width, top, height, str(player_label)]
        sub_video.append(row)
  sub_video = pd.DataFrame(sub_video, columns= columns)
  print('-------------------rmoving duplicates--------------------------')
  has_duplicate_labels = sub_video[["video_frame", "label"]].duplicated().any()
  #print(has_duplicate_labels)
  if has_duplicate_labels:
    a=sub_video[["video_frame", "label"]].duplicated()
    #print(sub_video[a])
    sub_video=sub_video.drop_duplicates(subset=["video_frame", "label"])
  #print(sub_video)
  has_duplicate_labels = sub_video[["video_frame", "label"]].duplicated().any()
  #print('a', has_duplicate_labels)
  
  print('--------------------removing frames with too much players-------------------------')
  remove_extra_helmets = []
  all_different_frames = sub_video['video_frame'].unique()
  #print(all_different_frames)
  for idx, vf in enumerate(all_different_frames):
    num_helms = len(sub_video[sub_video['video_frame'] == vf])
    if num_helms > 22:
      remove_extra_helmets.append(sub_video[sub_video['video_frame'] == vf].head(22))
      print(vf, num_helms)
    else:
      remove_extra_helmets.append(sub_video[sub_video['video_frame'] == vf])

  sub_video = pd.concat(remove_extra_helmets)
  sub_video = sub_video.reset_index()
  submission_df.append(sub_video)



submission_df = pd.DataFrame(submission_df, columns = header)
display(submission_df)

In [ ]:
columns = ['video_frame', 'left', 'width', 'top', 'height', 'label']
sub_video = []
for cluster in helmets_deepsort_one_vid['deepsort_cluster'].unique():
    deep_sort_cluster_data = helmets_deepsort_one_vid[helmets_deepsort_one_vid['deepsort_cluster'] == cluster].reset_index()
    player_label = Majority_voting_result[Majority_voting_result['cluster'] == cluster].reset_index()['PlayerID'][0]
    for idx, vf in enumerate(deep_sort_cluster_data['video_frame']):
        left = deep_sort_cluster_data['left'][idx]
        width= deep_sort_cluster_data['width'][idx]
        top = deep_sort_cluster_data['top'][idx]
        height = deep_sort_cluster_data['height'][idx]
        row = [vf, left, width, top, height, str(player_label)]
        sub_video.append(row)
sub_video = pd.DataFrame(sub_video, columns= columns)
print('-------------------rmoving duplicates--------------------------')
has_duplicate_labels = sub_video[["video_frame", "label"]].duplicated().any()
#print(has_duplicate_labels)
if has_duplicate_labels:
    a=sub_video[["video_frame", "label"]].duplicated()
    #print(sub_video[a])
    sub_video=sub_video.drop_duplicates(subset=["video_frame", "label"])
    #print(sub_video)
has_duplicate_labels = sub_video[["video_frame", "label"]].duplicated().any()
#print('a', has_duplicate_labels)

print('--------------------removing frames with too much players-------------------------')
remove_extra_helmets = []
all_different_frames = sub_video['video_frame'].unique()
#print(all_different_frames)
for idx, vf in enumerate(all_different_frames):
    num_helms = len(sub_video[sub_video['video_frame'] == vf])
    if num_helms > 22:
      remove_extra_helmets.append(sub_video[sub_video['video_frame'] == vf].head(22))
      print(vf, num_helms)
    else:
      remove_extra_helmets.append(sub_video[sub_video['video_frame'] == vf])

sub_video = pd.concat(remove_extra_helmets)
sub_video = sub_video.reset_index()
submission_df.append(sub_video)

submission_df = pd.DataFrame(submission_df, columns = header)
display(submission_df)

In [ ]:
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
labels = add_cols(labels)
display(labels)
sample_videos = submission_df['video'].unique())):
labels = labels[labels['video'].isin(sample_videos)]
scorer = NFLAssignmentScorer(labels)
baseline_score = scorer.score(submission_df)
print(f"validation score {baseline_score:0.4f}")

In [ ]:
# sub_df = submission_df[np.invert(submission_df[["video_frame", "label"]].duplicated())]
# print(len(sub_df))
# len(submission_df)

In [ ]:
def clean_submission(submission_df):

  submission_df = submission_df[np.invert(submission_df[["video_frame", "label"]].duplicated())]

  submission_df_with_extra_helms_removed = []
  for vf in submission_df['video_frame'].unique():
    num_helms = submission_df[submission_df['video_frame'] == vf]

    if len(num_helms) > 22:
      print('Video frame',vf,'has', len(num_helms) ,'frames')
      num_helms = num_helms.head(22)
      submission_df_with_extra_helms_removed.append(num_helms)
    else:
      submission_df_with_extra_helms_removed.append(num_helms)

  submission_df = pd.concat(submission_df_with_extra_helms_removed, ignore_index= True)

  return submission_df

In [ ]:
print("Length before cleaning:",len(submission_df))
submission_df = clean_submission(submission_df)
print("Length after cleaning:",len(submission_df))

submission_df.to_csv('submission.csv')

## Submission Accuracy

In [ ]:
import pandas as pd
ds = pd.read_csv("../input/deepsort-clusters5/deepsort_clusters.csv")
ds

In [ ]:
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
labels = labels[labels['video'].isin(['57906_00718_Endzone.mp4'])]
display(labels)




In [ ]:
# temp = '57781_000252_Sideline.mp4'
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
labels = add_cols(labels)
display(labels)


if debug:
  labels = labels[labels['video'].isin(sample_videos)]
  scorer = NFLAssignmentScorer(labels)
  baseline_score = scorer.score(submission_df)
  print(f"validation score {baseline_score:0.4f}")

In [ ]:
display(sample_videos)

In [ ]:
submission_df

In [ ]:
# #@input: tracking_data: full_track_information modified, so each row has frame and each frame has 22 players ID
# #       helmet_cluster: simple helmet cluster
# #@return: dataframe which matches the clusters to the players
# #         tracking_data: removed the player that have already been assigned and just removed on the frames where they have been assigned
# #         helmet_cluster: remove clusters that have been assigned already
# def big_cluste_assignement(track_data, helmet_cluster)


# input:table with ID-cluster
#     range clusters
#     tracking_data
#     helmet_cluster
    
#     return id_cluster updated